In [101]:
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib qt

## Setup



In [2]:
V=100
L=100
eps=1e-5

In [99]:
def get_grid():
    return np.zeros((2*L,2*L))

def set_boundry(grid):
    # grid[-1,:] = np.linspace(0, V ,2*L)
    # grid[:,-1] = np.linspace(0, V,2*L)

    grid[L, L-1:] = np.linspace(V, 0, L+1)
    # grid[L, L-1:] = 0
    grid[0:L+1,L-1] = np.linspace(0, V, L+1)
    # grid[0:L+1,L-1] = 0
    grid[0:L,L:] = np.zeros((L,L))

    # grid[L:,-1]=np.linspace(0, V ,L)
    # grid[-1,L:]=np.linspace(0, V ,L)

def compute_grid():
    N=0
    err = eps+1

    grid = get_grid()
    last_grid = get_grid()
    set_boundry(last_grid)

    while (err>eps):
        N+=1
        grid = get_grid()
        grid[1:-1,1:-1] = 0.25*(last_grid[2:,2:]+last_grid[2:,:-2]+last_grid[:-2,2:]+last_grid[:-2,:-2])
        set_boundry(grid)

        err = (grid-last_grid).max()
        last_grid = grid

    print(N)
    return grid

In [107]:
start = time.time()
grid1 = compute_grid()
print(time.time() - start)
# plt.imshow(grid1)

16813
3.1525352001190186


1682321597.734146

In [50]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

X = np.arange(2*L)
Y = np.arange(2*L)[::-1]
X, Y = np.meshgrid(X, Y)
# Z = grid + np.rot90(grid.T,2)
# Z = grid1+grid2
Z = grid
ax.contour(X, Y, Z, 50)

In [57]:
grid2 = grid-grid1
grid2[0:L,L:] = np.zeros((L,L))
grid2.min()

0.0

In [66]:
a = np.ones(2)
np.concatenate((a,np.ones(5)))

array([1., 1., 1., 1., 1., 1., 1.])

In [94]:
# def transform()

N = 10
D = 100

x = np.array([])
y = np.array([])

for i in np.linspace(0, 0.5, N):
    x = np.concatenate((x, np.linspace(i,1, D)))
    y = np.concatenate((y, np.ones(D)*i))
    y = np.concatenate((y, np.linspace(i,1, D)))
    x = np.concatenate((x, np.ones(D)*i))

for i in range(len(x)):
    xi = x[i]
    yi = y[i]

    v = yi/xi

    if 1 >= v >= 0.5:
        x[i] = 2*v*xi
        y[i] = 2*v*yi
    elif 1 < v <= 2:
        x[i] = 2*xi/v
        y[i] = 2*yi/v

fig, ax = plt.subplots()
ax.plot(x, y, 'o', color='lightgrey')



/var/folders/jn/78bt2nn54tl2wjjmmmcq84k40000gn/T/ipykernel_93943/3665491763.py:19: RuntimeWarning: invalid value encountered in double_scalars
  v = yi/xi
/var/folders/jn/78bt2nn54tl2wjjmmmcq84k40000gn/T/ipykernel_93943/3665491763.py:19: RuntimeWarning: divide by zero encountered in double_scalars
  v = yi/xi


In [138]:
def optimized_boundry(grid):
    grid[0, L-1:] = np.linspace(V, 0, L+1)
    grid[0, :L-1] = grid[:0:-1, L-1]

def optimized_compute():
    N=0
    err = eps+1

    grid = np.zeros((L, 2*L))
    last_grid = np.zeros((L, 2*L))
    optimized_boundry(last_grid)

    while (err>eps):
        N+=1
        grid = np.zeros((L, 2*L))
        grid[1:-1,1:-1] = 0.25*(last_grid[2:,2:]+last_grid[2:,:-2]+last_grid[:-2,2:]+last_grid[:-2,:-2])
        optimized_boundry(grid)

        err = (grid-last_grid).max()
        last_grid = grid

    print(N)
    return grid

In [141]:
start = time.time()
grid = optimized_compute()
print(time.time() - start)
plt.imshow(grid)

16813
1.826997995376587


In [114]:
arr = np.arange(4*2*4).reshape((4,8))
arr

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29, 30, 31]])

In [135]:
arr[-1:0:-1,3]

array([27, 19, 11])

In [132]:
arr[0,:3]

array([0, 1, 2])